In [1]:
import os
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Subset, Dataset, DataLoader, random_split

import pytorch_lightning as pl 
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback

from pytorch_forecasting.metrics import MAPE

In [ ]:
class Encoder(pl.LightningModule):
    def __init__(self, input_size, hidden_size, n_layers, dropout_p):
        super(Encoder, self).__init__()     
        self.rnn = nn.GRU(input_size = input_size, 
                          hidden_size = hidden_size, 
                          num_layers = n_layers, 
                          batch_first = True, 
                          dropout = dropout_p)
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_size).zero_()

    def forward(self, x):
        h_0 = self._init_state(batch_size=x.size(0))
        output, hidden = self.rnn(x, h_0)
        return output, hidden

In [ ]:
class Decoder(pl.LightningModule):
    def __init__(self, hidden_size, target_size):
        super(Decoder, self).__init__()
        self.out = nn.Linear(hidden_size, target_size)

    def forward(self, enc_output):
        output = enc_output[:, -1, :]
        pred = self.out(output)
        pred = pred.squeeze()
        return pred

In [ ]:
class Seq2seqLightningModule(pl.LightningModule):
    def __init__(self, hparams):
        super(Seq2seqLightningModule, self).__init__()
        self.hparams = hparams
        self.id_emb = nn.Embedding(self.hparams.n_id+1, self.hparams.embed_size)
        self.coin_emb = nn.Embedding(self.hparams.n_coin+1, self.hparams.embed_size)
        self.time_emb = nn.Embedding(self.hparams.n_time+1, self.hparams.embed_size)
        self.sec_emb = nn.Embedding(self.hparams.n_sec+1, self.hparams.embed_size)
        
        self.encoder = Encoder(self.hparams.embed_size, self.hparams.hidden_size, self.hparams.n_layers, self.hparams.dropout_p)
        self.decoder = Decoder(self.hparams.hidden_size, self.hparams.target_size)
        
        self.layer_norm = nn.LayerNorm(self.hparams.input_size)
        self.metric = MAPE()
        
    def forward(self, x_id, x_time, x_coin, x_open, x_sec):
        embed_x = self.id_emb(x_id) + self.time_emb(x_time) + self.coin_emb(x_coin) + self.sec_emb(x_sec) # [batch_size, input_dim, embed_size]
#         embed_x = torch.mean(embed_x, axis=-1) # [batch_size, input_dim]

        x_open = x_open.unsqueeze(1).repeat(1, 32, 1) # [batch_size, embed_dim, input_dim]
        
        x = torch.bmm(x_open, embed_x) # [batch_size, embed_dim, embed_dim]
        
#         x = F.relu(self.layer_norm(x))
        
        enc_output, _ = self.encoder(x)
        pred = self.decoder(enc_output)
        return pred


    def training_step(self, batch, batch_idx):
        x_id, x_time, x_coin, x_sec = batch['id'],  batch['time'], batch['coin'], batch['sec_type']
        x_open = batch['open_val']
        y = batch['labels']
        y_hat = self(x_id, x_time, x_coin, x_open, x_sec)
        loss = torch.mean(self.metric.loss(y_hat, y))
        self.log('trn_loss', loss, on_step=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x_id, x_time, x_coin, x_sec = batch['id'],  batch['time'], batch['coin'], batch['sec_type']
        x_open = batch['open_val']
        y = batch['labels']
        y_hat = self(x_id, x_time, x_coin, x_open, x_sec)
        loss = torch.mean(self.metric.loss(y_hat, y))
        self.log('val_loss', loss, on_step=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=5e-3)
        return optimizer

In [ ]:
n_id = 7362
n_coin = 10
n_sec = 149
n_time = 1499

In [ ]:
import easydict 

h_params = easydict.EasyDict({'n_id': n_id,
                              'n_sec': n_sec,
                              'n_time': n_time,
                              'n_coin': n_coin,
                              'input_size': 5,
                              'embed_size': 32,
                              'hidden_size': 64,
                              'target_size': 1,
                              'n_layers': 2,
                              'dropout_p': 0.2
                             })

In [ ]:
model = Seq2seqLightningModule(h_params)

In [ ]:
class BitcoinDataset(Dataset):
    def __init__(self, df, ws):
        self.df = df
        self.ws = ws
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        start_idx = index
        end_idx = index+self.ws
        
        sample_id = self.df.iloc[start_idx:end_idx, 0].values
        time = self.df.iloc[start_idx:end_idx, 1].values
        coin = self.df.iloc[start_idx:end_idx, 2].values
        open_val = self.df.iloc[start_idx:end_idx, 3].values
        sec_type = self.df.iloc[start_idx:end_idx, -2].values     
        labels = self.df.iloc[start_idx, -1]
        return {
            'id': torch.tensor(sample_id, dtype=torch.long),
            'time': torch.tensor(time, dtype=torch.long),
            'coin': torch.tensor(coin, dtype=torch.long),
            'open_val': torch.tensor(open_val, dtype=torch.float32),
            'sec_type': torch.tensor(sec_type, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.float32)
        }

In [ ]:
class BitcoinDataModule(pl.LightningDataModule):
    def __init__(self, window_size, batch_size):
        super().__init__()
        self.ws = window_size
        self.batch_size = batch_size
        
    def setup(self,stage=None):
        train_x = pd.read_csv('./Datasets/train_x_df.csv')
        train_y = pd.read_csv('./Datasets/train_y_df.csv')
        test = pd.read_csv('./Datasets/test_x_df.csv')
        
        # Feature Engineering
        train_y['time']+=1380
        train = pd.concat([train_x, train_y], axis=0)
        train = train.sort_values(by=['sample_id', 'time']).reset_index(drop=True)
        
        train['sec_type'] = train['time'].apply(lambda x: x//10)
        train['labels'] = train.groupby(['sample_id'])['open'].shift(-self.ws)
        
        test['sec_type'] = test['time'].apply(lambda x: x//10)
        test['labels'] = test.groupby(['sample_id'])['open'].shift(-self.ws)
        
        trn_subset_idx = train.loc[train['time']<1500-(self.ws)].index
        val_subset_idx = test.loc[test['time']<1380-(self.ws)].index
        
        trn_dataset=BitcoinDataset(train, ws=self.ws)
        val_dataset=BitcoinDataset(test, ws=self.ws)
        
        self.train_dataset = Subset(trn_dataset, trn_subset_idx)
        self.valid_dataset = Subset(val_dataset, val_subset_idx)
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(self.valid_dataset,
                          batch_size=self.batch_size,
                          num_workers=4)

In [ ]:
bitcoin_dm = BitcoinDataModule(5, 1024)

In [ ]:
bitcoin_dm.setup()

In [ ]:
model_checkpoint = ModelCheckpoint(monitor = "val_loss",
                                   verbose=True,
                                   filename="{epoch}_{val_loss:.4f}")

early_stop_callback = EarlyStopping(monitor='val_loss', 
                                    patience=2, 
                                    verbose=True, 
                                    mode='min')

In [ ]:
SEED = 42
EPOCHS = 2
device = "cuda" if torch.cuda.is_available() else "cpu"
pl.seed_everything(SEED)

In [ ]:
trainer = pl.Trainer(gpus=1, 
                     max_epochs=EPOCHS,
                     callbacks=[model_checkpoint, early_stop_callback]
                    )

In [ ]:
trainer.fit(model, bitcoin_dm)

In [ ]:
MODEL_PATH = './lightning_logs/version_9/checkpoints/epoch=0_val_loss=0.0298.ckpt'
HPARAM_PATH = './lightning_logs/version_7/hparams.yaml'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device='cpu'

In [ ]:
model = Seq2seqLightningModule.load_from_checkpoint(checkpoint_path=MODEL_PATH,
                                                    hparams_file = HPARAM_PATH)
model = model.to(device)
model.eval()
model.freeze()

In [ ]:
test_x = pd.read_csv('./Datasets/test_x_df.csv')
test_x['sec_type'] = test_x['time'].apply(lambda x: x//10)
test_x = test_x.groupby('sample_id').tail(5)
test_x = test_x.reset_index(drop=True)

In [ ]:
class BitcoinTestDataset(Dataset):
    def __init__(self, df, ws):
        self.df = df
        self.ws = ws
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        start_idx = index
        end_idx = index+self.ws
        
        sample_id = self.df.iloc[start_idx:end_idx, 0].values
        time = self.df.iloc[start_idx:end_idx, 1].values
        coin = self.df.iloc[start_idx:end_idx, 2].values
        open_val = self.df.iloc[start_idx:end_idx, 3].values
        sec_type = self.df.iloc[start_idx:end_idx, -1].values
        return {
            'id': torch.tensor(sample_id, dtype=torch.long),
            'time': torch.tensor(time, dtype=torch.long),
            'coin': torch.tensor(coin, dtype=torch.long),
            'sec_type': torch.tensor(sec_type, dtype=torch.long),
            'open_val': torch.tensor(open_val, dtype=torch.float32),
        }

In [ ]:
test_ds = BitcoinTestDataset(test_x, 5)
test_subset_idx = [i for i in test_x.index if i%5==0]
        
test_dataset = Subset(test_ds, test_subset_idx)

In [ ]:
test_dataset[0]

In [ ]:
test_dataloader= DataLoader(test_dataset,
                            batch_size=1,
                            num_workers=1)

In [ ]:
torch.rand(0)

In [ ]:
from tqdm.auto import tqdm
from collections import defaultdict 

# a = np.triu(np.ones([5,5]), k=1)[:-1][::-1]
# b = np.ones([1,5])
# mask = torch.tensor(np.concatenate((a,b)), dtype=torch.long).to(device)
# mask = mask.repeat(24,1)

result = defaultdict(list)

t = torch.tensor([i for i in range(1376, 1500)])
s = []
for i in range(137, 150):
    s.extend([i]*10)
s = torch.tensor(s[5:])  

idx = 0
for step, batch in enumerate(tqdm(test_dataloader)):
    x_id =  batch['id'].to(device)
    x_time =  batch['time'].to(device)
    x_coin = batch['coin'].to(device)
    x_sec = batch['sec_type'].to(device)
    x_open = batch['open_val'].to(device)
    y_hat = model(x_id, x_time, x_coin, x_open, x_sec)
    result[idx].append((y_hat.cpu().detach().numpy().tolist()))

    x_open = torch.cat((x_open[0][1:].contiguous(), y_hat.unsqueeze(0)), dim=0).unsqueeze(0).to(device)
    for i in range(119):
        x_time = t[i:i+5].unsqueeze(0)
        x_sec = s[i:i+5].unsqueeze(0)
        pred = model(x_id, x_time, x_coin, x_open, x_sec)
        result[idx].append(pred.cpu().detach().numpy().tolist())
        pred += torch.rand(1).item()
        x_open = torch.cat((x_open[0][1:].contiguous(), pred.unsqueeze(0)), dim=0).unsqueeze(0).to(device)
    idx+=1

In [ ]:
result_df = pd.DataFrame(result).transpose()

In [ ]:
result_df

In [ ]:
def array_to_submission(pred_array):
    # 입력 x_arrry와 출력 pred_arry를 통해서 
    # buy_quantitiy와 sell_time을 결정
    submission = pd.DataFrame(np.zeros([pred_array.shape[0],2], np.int64),
                              columns = ['buy_quantity', 'sell_time'])
    submission = submission.reset_index()
    submission.loc[:, 'buy_quantity'] = 0.1
    
    buy_price = []
    for idx, sell_time in enumerate(np.argmax(pred_array, axis = 1)):
        buy_price.append(pred_array[idx, sell_time])
    buy_price = np.array(buy_price)
    # 115% 이상 상승한하고 예측한 sample에 대해서만 100% 매수
    submission.loc[:, 'buy_quantity'] = (buy_price > 1.15) * 1
    # 모델이 예측값 중 최대 값에 해당하는 시간에 매도
    submission['sell_time'] = np.argmax(pred_array, axis = 1)
    submission.columns = ['sample_id','buy_quantity', 'sell_time']
    return submission

In [ ]:
sub = array_to_submission(result_df.values)

In [ ]:
sub['buy_quantity']=0.5

In [ ]:
sub.to_csv('./test.csv', index=False)

In [ ]:
submission = pd.read_csv('./Datasets/sample_submission.csv')

In [ ]:
submission.loc[:, 'buy_quantity'] = 0.1

In [ ]:
sub.info()